In [ ]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes

In [ ]:
!nvidia-smi

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 3072 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/mistral-7b-bnb-4bit",
    "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
    "unsloth/llama-2-7b-bnb-4bit",
    "unsloth/gemma-7b-bnb-4bit",
    "unsloth/gemma-7b-it-bnb-4bit", # Instruct version of Gemma 7b
    "unsloth/gemma-2b-bnb-4bit",
    "unsloth/gemma-2b-it-bnb-4bit", # Instruct version of Gemma 2b
    "unsloth/llama-3-8b-bnb-4bit", # [NEW] 15 Trillion token Llama-3
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    # model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    cache_dir = "my_cache"
    # use_gradient_checkpointing = True
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

In [ ]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
llama3_system = "You are a state-of-the-art AI assistant named TestGenBot. Your task is to understand the use case of a module/function and create thorugh test cases for that partiuclar use case."
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN

def formatting_prompts_func(examples):
    prompts = examples["prompt"]
    responses  = examples["response"]
    texts = []

    texts = []
    for prompt, response in zip(prompts, responses):
        messages = [
            # {"role": "system", "content": llama3_system},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content":  response}
        ]

        prompt = tokenizer.apply_chat_template(
                messages,
                tokenize=False,
                # add_generation_prompt=True
        )
        texts.append(prompt)
    return { "text" : texts, }

In [ ]:
from datasets import load_dataset, Dataset
import random

# Set the seed for reproducibility
random.seed(53)

dataset = Dataset.load_from_disk("final-train-dataset")

# Split the dataset into training and validation sets
dataset = dataset.train_test_split(test_size=0.1, shuffle=False)

# Check the structure of the dataset
print(dataset)


In [ ]:
dataset['train'][50]

In [ ]:
# INSTRUCTION = dataset["train"][0]["instruction"]

# def new_instruction(example):
#     global INSTRUCTION
#     example["prompt"] = INSTRUCTION + "\n\n" + example["prompt"]
#     return example


# # dataset = dataset.map(new_instruction)
# dataset_temp = dataset.map(new_instruction)

In [ ]:
dataset = dataset.map(formatting_prompts_func, batched=True)

In [ ]:
dataset['train'][50]

In [ ]:
dataset_train = dataset["train"]
dataset_eval = dataset["test"]

In [ ]:
def formatting_prompts_func(example):
    return example["text"]

In [ ]:

response_template = "<|start_header_id|>assistant<|end_header_id|>"
collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)

In [ ]:
NUM_TRAIN_EPOCHS = 10

In [ ]:

from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset_train,
    eval_dataset = dataset_eval,
    formatting_func=formatting_prompts_func,
    data_collator=collator,
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.05,
        num_train_epochs = NUM_TRAIN_EPOCHS,
        learning_rate = 2e-5,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_strategy="steps",
        optim = "adamw_8bit",
        logging_steps=10,  # Log loss every 10 steps
        weight_decay = 0.1,
        lr_scheduler_type = "cosine",
        seed = 53,
        output_dir = "outputs",
        fp16_full_eval = True,
        per_device_eval_batch_size = 4,
        eval_accumulation_steps = 4,
        eval_strategy = "steps",
        eval_steps = 50,
        save_strategy="steps",
        save_total_limit=2,
        load_best_model_at_end=True
    ),
)

In [ ]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [ ]:
trainer_stats = trainer.train()

In [ ]:
NAME = f"uc_tc_mistral_0.3_7b_instruct_{NUM_TRAIN_EPOCHS}epochs_final"
model.save_pretrained(NAME)
tokenizer.save_pretrained(NAME)

In [ ]:
import gc

torch.cuda.empty_cache()                    # Clear memory
del model, tokenizer, trainer, trainer_stats # Delete objects
gc.collect()                                # Empty trash